# Visualize Option Prices

In [37]:
import pandas as pd
import yfinance as yf


class OptionPrices:

    def __init__(self, ticker):
        self.ticker = ticker
        self.get_option_chains()
        self.get_prices()

    def get_option_chains(self):
        asset = yf.Ticker(self.ticker)
        self.expirations = asset.options
        self.call_chain = {
            expiration: asset.option_chain(expiration).calls
            for expiration in self.expirations
        }
        self.put_chain = {
            expiration: asset.option_chain(expiration).puts
            for expiration in self.expirations
        }

    def get_prices(self, price_estimate="avg"):
        self.calls = self.estimate_option_prices(
            self.call_chain,
            option_type="call",
            price_estimate=price_estimate
        )
        self.puts = self.estimate_option_prices(
            self.put_chain,
            option_type="put",
            price_estimate=price_estimate
        )

    def estimate_option_prices(self, raw_option_chain, option_type, price_estimate):
        option_chain = {}
        for expiration, raw_options in raw_option_chain.items():
            options = pd.DataFrame(raw_options["contractSymbol"])
            options["strikePrice"] = raw_options["strike"]

            if price_estimate == "avg":
                options["optionPrice"] = (raw_options.bid + raw_options.ask) / 2
            else:
                options["optionPrice"] = raw_options.lastPrice

            if option_type == "call":
                options["profitPrice"] = options.strikePrice + options.optionPrice
            else:
                options["profitPrice"] = options.strikePrice - options.optionPrice

            option_chain[expiration] = options
        return option_chain

ticker = "TQQQ"
opt = OptionPrices(ticker)


In [42]:
opt.calls[opt.expirations[-1]]

,contractSymbol,strikePrice,optionPrice,profitPrice
0,TQQQ270115C00035000,35.0,58.450,93.450
1,TQQQ270115C00040000,40.0,55.950,95.950
2,TQQQ270115C00045000,45.0,52.600,97.600
3,TQQQ270115C00050000,50.0,49.600,99.600
4,TQQQ270115C00055000,55.0,45.500,100.500
5,TQQQ270115C00056000,56.0,45.000,101.000
6,TQQQ270115C00057000,57.0,43.600,100.600
7,TQQQ270115C00058000,58.0,43.975,101.975
8,TQQQ270115C00059000,59.0,43.000,102.000
9,TQQQ270115C00060000,60.0,42.500,102.500


In [43]:
opt.puts[opt.expirations[-1]]

,contractSymbol,strikePrice,optionPrice,profitPrice
0,TQQQ270115P00035000,35.0,5.050,29.950
1,TQQQ270115P00040000,40.0,6.925,33.075
2,TQQQ270115P00045000,45.0,7.975,37.025
3,TQQQ270115P00050000,50.0,9.800,40.200
4,TQQQ270115P00055000,55.0,11.625,43.375
5,TQQQ270115P00056000,56.0,11.000,45.000
6,TQQQ270115P00057000,57.0,11.750,45.250
7,TQQQ270115P00058000,58.0,12.700,45.300
8,TQQQ270115P00059000,59.0,13.100,45.900
9,TQQQ270115P00060000,60.0,13.375,46.625
